In [1]:
# !pip3 install pydub
# !pip3 install SpeechRecognition

In [2]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_nonsilent
import os

In [3]:
import speech_recognition as sr
import os
import uuid

r = sr.Recognizer()

In [4]:
def audiosegment_google_speech(audio, filename):
    if os.path.exists('output-wav/' + filename):
        return False
    
    audio.export(filename, format="wav")
    try:
        with sr.AudioFile(filename) as source:
            audio = r.record(source)

        text = r.recognize_google(audio,language="ms")
    except:
        text = ''
        
    if len(text):
        text_filename = f'output-text/{filename}.txt'
        with open(text_filename, 'w') as fopen:
            fopen.write(text)
            
        os.replace(filename, 'output-wav/' + filename)
        
    else:
        os.remove(filename)
    
    return True

In [5]:
def split(file, min_silence_len=50, silence_thresh=-20, max_len = 10, keep_silence = 500):
    audio = AudioSegment.from_mp3(file)
    audio_chunks = split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh,
                                   keep_silence = keep_silence)
    
    audios, temp, length = [], [], 0
    for i in range(len(audio_chunks)):
        if length + audio_chunks[i].duration_seconds >= max_len and len(temp):
            audios.append(sum(temp))
            temp = []
            length = 0
        temp.append(audio_chunks[i])
        length += audio_chunks[i].duration_seconds

    if len(temp):
        audios.append(sum(temp))
        
    return audios, audio

In [6]:
# !mkdir output-wav
# !mkdir output-text

In [7]:
from boto3 import client

conn = client('s3')
contents = conn.list_objects(Bucket='huseinhouse-storage',Prefix='speech/')['Contents']
contents = [f for f in contents if '.mp3' in f['Key']]
contents = sorted(contents, key=lambda k: k['Size']) 

In [9]:
import urllib.parse
from tqdm import tqdm

for c in contents:
    key = c['Key']
    filename = urllib.parse.unquote(os.path.split(key)[1])
    filename = filename.replace(' ', '-')
    
    if os.path.exists(filename):
        continue
        
    if 'DrAmerSiddiqonMentalHealth' in filename:
        continue
        
    url = f'https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/{urllib.parse.quote(key)}'
    os.system(f'wget {url} -O {filename}')
    
    try:
    
        audios, audio = split(filename)
        audios = audios[1:]
        print(filename, key, url, len(audios))

        for part in tqdm(range(len(audios))):
            temp_filename = f'{filename}-part-{part}.wav'
            audiosegment_google_speech(audios[part], temp_filename)
    except:
        pass

  0%|          | 0/41 [00:00<?, ?it/s]

HM_27_Feb_2019.mp3 speech/HM_27_Feb_2019.mp3 https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/speech/HM_27_Feb_2019.mp3 41


  0%|          | 0/43 [00:00<?, ?it/s]

SH1_18_Feb_2019.mp3 speech/SH1_18_Feb_2019.mp3 https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/speech/SH1_18_Feb_2019.mp3 43


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/395 [00:00<?, ?it/s]

Mas-Idayu-Komen-Nyanyian-Muaz--Pedofil-Dapat-Habuan.--Sekolah-Paling-Mahal-Di-Dunia_290819.mp3 speech/Mas%20Idayu%20Komen%20Nyanyian%20Muaz%20%20Pedofil%20Dapat%20Habuan.%20%20Sekolah%20Paling%20Mahal%20Di%20Dunia_290819.mp3 https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/speech/Mas%2520Idayu%2520Komen%2520Nyanyian%2520Muaz%2520%2520Pedofil%2520Dapat%2520Habuan.%2520%2520Sekolah%2520Paling%2520Mahal%2520Di%2520Dunia_290819.mp3 395


  0%|          | 0/303 [00:00<?, ?it/s]

Remaja-Dan-Seks---Apa-Peranan-Masyarakat.--RM10k-Buru-Penyamun-MRT.--Ada-Tak-Vaksin-Untuk-Orang-Bodoh_180219.mp3 speech/Remaja%20Dan%20Seks%20-%20Apa%20Peranan%20Masyarakat.%20%20RM10k%20Buru%20Penyamun%20MRT.%20%20Ada%20Tak%20Vaksin%20Untuk%20Orang%20Bodoh_180219.mp3 https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/speech/Remaja%2520Dan%2520Seks%2520-%2520Apa%2520Peranan%2520Masyarakat.%2520%2520RM10k%2520Buru%2520Penyamun%2520MRT.%2520%2520Ada%2520Tak%2520Vaksin%2520Untuk%2520Orang%2520Bodoh_180219.mp3 303


100%|██████████| 303/303 [12:53<00:00,  2.55s/it]
